In [38]:
from sklearn.neighbors import KNeighborsClassifier
import cv2
import os
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tqdm
from random import shuffle

In [39]:
with open ("train_ids.json", 'r') as f:
    train_ids = json.load(f)
with open ("val_ids.json", 'r') as f:
    val_ids = json.load(f)

    
shuffle(train_ids)
shuffle(val_ids)

print(len(train_ids))
print(len(val_ids))


12527
1283


In [3]:
# # temp, remove those for which cutouts are not ready yet
# to_remove_train = []
# for pid in train_ids:
#     image_name = "P"+ str(pid).zfill(6)+".jpg"
#     if not os.path.isfile("data/train_images/"+image_name):
#         to_remove_train.append(pid)
# print(len(to_remove_train))


# to_remove_val = []
# for pid in val_ids:
#     image_name = "P"+ str(pid).zfill(6)+".jpg"
#     if not os.path.isfile("data/val_images/"+image_name):
#         to_remove_val.append(pid)
# print(len(to_remove_val))


# train_ids = list(set(train_ids) - set(to_remove_train))
# val_ids = list(set(val_ids) - set(to_remove_val))
# print(len(train_ids))
# print(len(val_ids))

In [40]:
df = pd.read_csv('../full_data/expanded_catalogue.csv')  
print(df.shape)



/tmp/ipykernel_65632/1658725802.py:1: DtypeWarning: Columns (4,5,45,46,62,64,68,69,70,71,73,76,77,78,79,80,81,83,84,88,90,91,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../full_data/expanded_catalogue.csv')


(133923, 99)


In [41]:
# preprocess images and prepare labels
# run resize once, next time read from train_images_resized and val_images_resized
Y_train_label = [] # this has label name
Y_val_label = []
Y_train = [] # this has label value (encoded via Label Encoder)
Y_val = []
X_train = []
X_val = []


size = 64

for pid in tqdm.tqdm(train_ids):
    Y_train_label.append(df.loc[df['id'] ==pid]['period.period'].values[0])
    image_name = "P"+ str(pid).zfill(6)+".jpg"
    image = cv2.imread("data/train/"+image_name)
    image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)
#     cv2.imwrite("train_images_resized/"+image_name, image)
    X_train.append(np.array(image))


for pid in tqdm.tqdm(val_ids):
    Y_val_label.append(df.loc[df['id'] ==pid]['period.period'].values[0])
    image_name = "P"+ str(pid).zfill(6)+".jpg"
    image = cv2.imread("data/val/"+image_name)
    image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)
#     cv2.imwrite("val_images_resized/"+image_name, image)
    X_val.append(np.array(image))

X_train = np.array(X_train)
X_val = np.array(X_val)
print(X_train.shape)
print(X_val.shape)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1283/1283 [00:07<00:00, 162.96it/s]


(12527, 64, 64, 3)
(1283, 64, 64, 3)


In [42]:
del df

In [43]:

def flattenArray(images):
    imgs = []
    for img in images:
        imgs.append(np.array(img).flatten())
    return np.array(imgs)

In [44]:
Y_train = np.array(Y_train_label)
le = LabelEncoder()
Y_train = le.fit_transform(Y_train_label)
Y_val = np.array(Y_val_label)
Y_val = le.transform(Y_val)
# le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print(le_name_mapping)

In [9]:
# from sklearn.decomposition import PCA, IncrementalPCA
# flatX_train = flattenArray(X_train)
# ipca = IncrementalPCA(copy=False, n_components=224, batch_size=10)
# X_train_ipca = ipca.fit_transform(flatX_train)




In [45]:
# change this to fit on train later
pca = PCA(n_components = 224)
X_train_pca = pca.fit_transform(flattenArray(X_train))
X_val_pca = pca.transform(flattenArray(X_val))
print(X_val_pca.shape)
print(X_train_pca.shape)

(1283, 224)
(12527, 224)


In [46]:
# change this to fit on train later
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train_pca, Y_train)

KNeighborsClassifier(n_neighbors=8)

In [47]:

Y_predicted_train = knn.predict(X_train_pca)
acc_train = accuracy_score(Y_train, Y_predicted_train)
Y_predicted_val = knn.predict(X_val_pca)
acc_val = accuracy_score(Y_val, Y_predicted_val)
print(acc_train, acc_val )

0.4610042308613395 0.2618862042088854


In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train, Y_predicted_train)

array([[ 69,   3,  26,   3,   6,   4,   2,  12,  20,   0,  10,  12,  21,
          7,   7,   5,   5,   0],
       [  4, 130, 103,  14,  23,   8,   2,   7,  10,   2,  12,  20,   8,
          9,   6,  18,   7,   1],
       [  6,  21, 727,  27,  27,  14,   0,  10,  23,   9,  50,  24,  12,
         16,  17,   8,   9,   0],
       [  4,  14,  52, 614,   5,  45,  11,   8,   8,   4,  22,  54,  29,
         13, 108,   6,   3,   0],
       [  5,  21, 111,  20, 306,   6,   1,   3,  10,   1,  11,   8,   7,
         12,   6,   6,   5,   1],
       [  7,  10,  58,  51,  12, 384,   1,   8,   6,   6,  16,  27,   7,
          9,  17,   6,   5,   0],
       [  4,   1,  14,  10,   3,   1,  58,   1,   6,   1,   5,  43,   4,
          5,  11,   0,   1,   0],
       [  5,   2,  40,  10,   3,   1,   7, 132,  13,   1,  26,  63,  12,
          8,  19,   2,   3,   0],
       [ 30,  22,  97,  51,  17,  19,   9,  63, 414,  10,  29,  93,  48,
         62,  17,  11,   8,   0],
       [  2,   4,  16,  15,   5,   6,

In [49]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=15, random_state=0)
clf.fit(X_train_pca, Y_train)

RandomForestClassifier(max_depth=15, random_state=0)

In [50]:

Y_predicted_train = clf.predict(X_train_pca)
acc_train = accuracy_score(Y_train, Y_predicted_train)
Y_predicted_val = clf.predict(X_val_pca)
acc_val = accuracy_score(Y_val, Y_predicted_val)
print(acc_train, acc_val )

0.9329448391474415 0.3047544816835542


In [33]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=18, random_state=0, n_init="auto").fit(X_train_pca)
Y_kmeans_train = kmeans.predict(X_train_pca)
Y_kmeans_val = kmeans.predict(X_val_pca)
Y_kmeans_val

array([14,  2, 14, ...,  9, 17, 15], dtype=int32)

In [52]:
Y_val

array([13, 16, 13, ..., 12, 14, 12])

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='linear',gamma='auto')
svc.fit(X_train_pca, Y_train)

In [ ]:

Y_predicted_train = knn.predict(X_train_pca)
acc_train = accuracy_score(Y_train, Y_predicted_train)
Y_predicted_val = knn.predict(X_val_pca)
acc_val = accuracy_score(Y_val, Y_predicted_val)
print(acc_train, acc_val )